In [17]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("Uber Request Data.csv")
df.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Request id         6745 non-null   int64  
 1   Pickup point       6745 non-null   object 
 2   Driver id          4095 non-null   float64
 3   Status             6745 non-null   object 
 4   Request timestamp  6745 non-null   object 
 5   Drop timestamp     2831 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 316.3+ KB


In [19]:
df.describe(include='all')

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
count,6745.000000,6745,4095.000000,6745,6745,2831
unique,NaN,2,NaN,3,5618,2598
top,NaN,City,NaN,Trip Completed,11/7/2016 19:02,11/7/2016 13:00
freq,NaN,3507,NaN,2831,6,4
mean,3384.644922,NaN,149.501343,NaN,NaN,NaN
std,1955.099667,NaN,86.051994,NaN,NaN,NaN
min,1.000000,NaN,1.000000,NaN,NaN,NaN
25%,1691.000000,NaN,75.000000,NaN,NaN,NaN
50%,3387.000000,NaN,149.000000,NaN,NaN,NaN
75%,5080.000000,NaN,224.000000,NaN,NaN,NaN


In [20]:
df.isnull().sum()

Request id              0
Pickup point            0
Driver id            2650
Status                  0
Request timestamp       0
Drop timestamp       3914
dtype: int64

In [21]:
df.nunique()

Request id           6745
Pickup point            2
Driver id             300
Status                  3
Request timestamp    5618
Drop timestamp       2598
dtype: int64

In [22]:
print(df['Status'].value_counts())
print(df['Pickup point'].value_counts())

Status
Trip Completed       2831
No Cars Available    2650
Cancelled            1264
Name: count, dtype: int64
Pickup point
City       3507
Airport    3238
Name: count, dtype: int64


In [40]:
df['Request timestamp'] = pd.to_datetime(df['Request timestamp'], dayfirst=True, errors='coerce')
df['Drop timestamp'] = pd.to_datetime(df['Drop timestamp'], dayfirst=True, errors='coerce')

In [24]:
df['request_hour'] = df['Request timestamp'].dt.hour
df['request_day'] = df['Request timestamp'].dt.day_name()
df[['Request timestamp', 'request_hour', 'request_day']].head()

,Request timestamp,request_hour,request_day
0,2016-07-11 11:51:00,11.0,Monday
1,2016-07-11 17:57:00,17.0,Monday
2,2016-07-12 09:17:00,9.0,Tuesday
3,2016-07-12 21:08:00,21.0,Tuesday
4,NaT,NaN,NaN


In [25]:
hourly_status = df.groupby(['request_hour', 'Status']).size().unstack(fill_value=0)
hourly_status

Status,Cancelled,No Cars Available,Trip Completed
request_hour,,,
0.0,1,18,13
1.0,3,17,8
2.0,2,21,7
3.0,1,18,12
4.0,14,31,37
5.0,66,30,75
6.0,64,35,72
7.0,64,16,70
8.0,64,37,58


In [26]:
completed_trips_by_driver = df[df['Status'] == 'Trip Completed'].groupby('Driver id').size().sort_values(ascending=False)
completed_trips_by_driver.head()

Driver id
22.0     16
233.0    15
184.0    15
126.0    14
107.0    14
dtype: int64

In [27]:
df['trip_duration_minutes'] = (df['Drop timestamp'] - df['Request timestamp']).dt.total_seconds() / 60
df[df['Status'] == 'Trip Completed']['trip_duration_minutes'].describe()

count    1150.000000
mean       52.470435
std        13.762707
min        21.000000
25%        41.000000
50%        52.000000
75%        64.000000
max        83.000000
Name: trip_duration_minutes, dtype: float64